In [1]:
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg -y
!sudo apt-get install portaudio19-dev # Linux-case
!python -m pip install pyaudio
!pip install SpeechRecognition
!pip install pyttsx3
!pip install --upgrade pyttsx3
!pip uninstall pyttsx3
!pip install pyttsx3==2.90
!sudo apt-get install -y espeak
!sudo apt-get update

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-gts8es2h
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-gts8es2h
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpa

In [2]:
from scipy.io import wavfile
import numpy as np
import soundfile as sf
import noisereduce as nr
import speech_recognition as sr
import pyttsx3
from jiwer import wer

In [3]:
clean_audio = "/content/my_clean_audio.wav"
noisy_audio = "/content/my_noisy_audio.wav"
cleaned_audio = "/content/my_cleaned_audio.wav"

In [10]:
reference_sentence = "Hello everyone. This is background checking"

In [4]:
def apply_noise_reduction(input_file, output_file):
    audio, sr = sf.read(input_file)

    # If stereo, convert to mono
    if audio.ndim > 1:
        audio = np.mean(audio, axis=1)

    # Apply noise reduction
    reduced_audio = nr.reduce_noise(y=audio, sr=sr)

    # Save cleaned file
    sf.write(output_file, reduced_audio, sr)
    print(f"Noise-reduced audio saved as: {output_file}")

In [5]:
apply_noise_reduction("my_noisy_audio.wav", "my_cleaned_audio.wav")

Noise-reduced audio saved as: my_cleaned_audio.wav


In [6]:
def transcribe_audio(file_path):
    recognizer = sr.Recognizer()
    with sr.AudioFile(file_path) as source:
        audio = recognizer.record(source)  # read entire file
    try:
        text = recognizer.recognize_google(audio)
    except sr.UnknownValueError:
        text = "[Unrecognized Speech]"
    except sr.RequestError:
        text = "[API unavailable]"
    return text

In [7]:
clean_text = transcribe_audio(clean_audio)
noisy_text = transcribe_audio(noisy_audio)
cleaned_text = transcribe_audio(cleaned_audio)

In [11]:
print("Reference: ", reference_sentence)
print("Clean Recording: ", clean_text)
print("Noisy Recording: ", noisy_text)
print("Cleaned Recording: ", cleaned_text)

Reference:  Hello everyone. This is background checking
Clean Recording:  hello everyone can see this for background checking at it will i t s i
Noisy Recording:  hello everyone dressing at 8:00 DSi
Cleaned Recording:  studying at 8:00 PSI


In [12]:
wer_noisy = wer(reference_sentence.lower(), noisy_text.lower())
wer_cleaned = wer(reference_sentence.lower(), cleaned_text.lower())

In [13]:
print(f"WER (Noisy):   {wer_noisy*100:.2f}%")
print(f"WER (Cleaned): {wer_cleaned*100:.2f}%")

WER (Noisy):   83.33%
WER (Cleaned): 100.00%
